In [21]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense


In [13]:
import numpy as np
from matplotlib import pyplot as plt

In [14]:
import idx2numpy

In [15]:
import cv2

In [16]:
emnist_labels = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]

In [17]:
#модель нейронной сети
def model_for_text():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(28, 28, 1), activation='relu'))
   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same',input_shape=(28, 28, 1), activation='relu'))
   
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation="softmax"))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
    return model

In [50]:
def model_train(model): #обучение сети
    y_train= idx2numpy.convert_from_file('EMNIST/emnist_source_files/emnist-byclass-train-labels-idx1-ubyte') #обучение
    x_train= idx2numpy.convert_from_file('EMNIST/emnist_source_files/emnist-byclass-train-images-idx3-ubyte')
    
    x_test= idx2numpy.convert_from_file('EMNIST/emnist_source_files/emnist-byclass-test-images-idx3-ubyte') #валидация
    y_test= idx2numpy.convert_from_file('EMNIST/emnist_source_files/emnist-byclass-test-labels-idx1-ubyte')
    
    x_train = np.reshape(x_train, (x_train.shape[0], 28, 28, 1))
    x_test = np.reshape(x_test, (x_test.shape[0], 28, 28, 1))
    
    x_train = x_train[:x_train.shape[0]//50] 
    y_train = y_train[:y_train.shape[0] //50]
    x_test = x_test[:x_test.shape[0] //50]
    y_test = y_test[:y_test.shape[0] //50]
    
    
   
    x_train = x_train.astype(np.float32) #norm
    x_train /= 255.0
    x_test = x_test.astype(np.float32)
    x_test /= 255.0
    
    
    y_train_categorical = keras.utils.to_categorical(y_train, len(emnist_labels)) 
    y_test_categorical = keras.utils.to_categorical(y_test, len(emnist_labels))
    
    
    model.fit(x_train, y_train_categorical, validation_split=0.2, validation_data=(x_test, y_test_categorical),  batch_size=64, epochs=40)
    model.save('model_from_emnist.h5')

In [51]:
model=model_for_text()
model_train(model)

Epoch 1/40
219/219 [==============================] - 17s 73ms/step - loss: 1.8918 - accuracy: 0.5157 - val_loss: 0.9204 - val_accuracy: 0.7231
Epoch 2/40
219/219 [==============================] - 15s 68ms/step - loss: 0.9869 - accuracy: 0.7001 - val_loss: 0.7200 - val_accuracy: 0.7670
Epoch 3/40
219/219 [==============================] - 15s 70ms/step - loss: 0.7842 - accuracy: 0.7444 - val_loss: 0.6462 - val_accuracy: 0.7889
Epoch 4/40
219/219 [==============================] - 15s 69ms/step - loss: 0.6868 - accuracy: 0.7684 - val_loss: 0.5904 - val_accuracy: 0.8044
Epoch 5/40
219/219 [==============================] - 14s 66ms/step - loss: 0.6204 - accuracy: 0.7870 - val_loss: 0.5652 - val_accuracy: 0.8087
Epoch 6/40
219/219 [==============================] - 14s 66ms/step - loss: 0.5708 - accuracy: 0.8002 - val_loss: 0.5544 - val_accuracy: 0.8100
Epoch 7/40
219/219 [==============================] - 16s 72ms/step - loss: 0.5381 - accuracy: 0.8099 - val_loss: 0.5666 - val_accuracy:

In [55]:
#подготовка изображения и получение контуров с подготовленноко изображения
def image_preparation(path):
  img = cv2.imread(path)

  gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#серый тон

  ret, tresh=cv2.threshold(gray,0,255,cv2.THRESH_BINARY)# черно-белый

  img_erode=cv2.erode(tresh, np.ones((3,3),np.uint8),iterations=1)#улучшение четкости

  contours, hierarchy=cv2.findContours(img_erode, cv2.RETR_TREE ,cv2.CHAIN_APPROX_SIMPLE)#получение контуров
  
  output = img.copy()
  letters = []
  for idx, contour  in enumerate(contours):
    (x, y, w, h) = cv2.boundingRect(contour)
    if hierarchy[0][idx][3] == 0:
      cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)
      
      letter_crop = gray[y:y + h, x:x + w]
      size_max = max(w, h)
      letter_square = 255 * np.ones(shape=[size_max, size_max], dtype=np.uint8)
      if w > h:
        
        y_pos = size_max//2 - h//2
        letter_square[y_pos:y_pos + h, 0:w] = letter_crop
      elif w < h:
        
        x_pos = size_max//2 - w//2
        letter_square[0:h, x_pos:x_pos + w] = letter_crop
      else:
        
        letter_square = letter_crop
      letters.append((x, w, cv2.resize(letter_square, (28,28), interpolation=cv2.INTER_AREA)))
    
    
  letters.sort(key=lambda x: x[0], reverse=False)
  #cv2.imshow("Output", output)
  #cv2.waitKey(0)
  return letters

In [56]:
def predict_img(model, letters): #поворот ихображения
    img_arr = np.expand_dims(letters, axis=0)
    img_arr = 1 - img_arr/255.0
    img_arr[0] = np.rot90(img_arr[0], 3)
    img_arr[0] = np.fliplr(img_arr[0])
    img_arr = img_arr.reshape((1, 28, 28, 1))

    predict = model.predict([img_arr])
    result = np.argmax(predict, axis=1)
    return chr(emnist_labels[result[0]])

In [57]:
def img_to_txt(model, img): #вывод текста
    letters = image_preparation(img)
    text = ""
    for i in range(len(letters)):
        dn = letters[i+1][0] - letters[i][0] - letters[i][1] if i < len(letters) - 1 else 0
        text += predict_img(model, letters[i][2])
        if (dn > letters[i][1]/4):
            text += ' '
    return text

In [62]:
model = keras.models.load_model('model_from_emnist.h5')
text = img_to_txt(model, "text_6.png")
#text = img_to_txt(model, "text_2.png")
#text = img_to_txt(model, "text_3.png")
#text = img_to_txt(model, "text_4.png")
#text = img_to_txt(model, "text_5.png")
#text = img_to_txt(model, "text_6.png")
print(text)

1/1 [==============================] - 0s 20ms/step
hMdTW
